# Delta-Neutral Arbitrage Parameter Optimization

This notebook provides interactive parameter optimization for delta-neutral arbitrage strategies using the methods from `cost_utils.py`.

## Overview

Delta-neutral arbitrage profits from temporary price discrepancies between correlated instruments (spot vs futures) while remaining hedged against directional price movements. The key challenge is optimizing:

- **Entry Threshold** (`max_entry_cost_pct`): The spread cost threshold for entering positions
- **Exit Target** (`min_profit_pct`): The profit target for closing positions

We'll compare multiple optimization approaches:
1. **Statistical Approach**: Based on historical spread distribution
2. **Risk-Adjusted Approach**: Optimized for Sharpe ratio and drawdown control
3. **Fast Statistical**: Quick approximation method
4. **Random Sampling**: Efficient parameter space exploration

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timezone, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.precision', 4)
pd.set_option('display.float_format', None)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📦 Libraries imported successfully")

📦 Libraries imported successfully


In [ ]:
# Import project modules
import sys
import os

# Fix Python path priority issue - insert at beginning to override conflicting paths
project_src_path = '/Users/dasein/dev/cex_arbitrage/src'
if project_src_path in sys.path:
    sys.path.remove(project_src_path)
sys.path.insert(0, project_src_path)

from exchanges.structs import Symbol, AssetName
from trading.research.cost_utils import (
    ParameterRecommendation,
    SpreadStatistics,
    calculate_spread_statistics,
    optimize_parameters_statistical,
    optimize_parameters_risk_adjusted,
    optimize_parameters_statistical_fast,
    optimize_parameters_random_sampling,
    compare_parameter_approaches,
    print_optimization_summary
)

# Import data loading function
from trading.research.mexc_gateio_spread_analysis import load_cached_data

print("✅ Project modules imported successfully")

## 1. Data Loading and Preparation

Load historical market data for parameter optimization analysis.

In [ ]:
# Configuration
symbol = Symbol(base=AssetName("F"), quote=AssetName("USDT"))
date_to = datetime.fromisoformat("2025-10-18 05:50:00").replace(tzinfo=timezone.utc)
date_from = datetime.fromisoformat("2025-10-17 21:50:00").replace(tzinfo=timezone.utc)

print(f"📊 Loading data for {symbol.base}/{symbol.quote}")
print(f"📅 Period: {date_from} to {date_to}")
print(f"⏱️  Duration: {(date_to - date_from).total_seconds() / 3600:.1f} hours")

# Load market data
df = load_cached_data(symbol, date_from, date_to)

print(f"✅ Loaded {len(df)} synchronized data points")
print(f"📈 Data range: {df.index.min()} to {df.index.max()}")

In [ ]:
# Display data summary
print("📋 Data Summary:")
print(f"  Columns: {list(df.columns)}")
print(f"  Shape: {df.shape}")
print(f"  Index type: {type(df.index)}")
print(f"  Missing values: {df.isnull().sum().sum()}")

# Show first few rows
display(df.head())

## 2. Fee Configuration

Define the fee structure for different exchanges and trading types.

In [ ]:
# Fee configuration (basis points)
FEES = {
    'mexc_spot': {'maker': 0.0002, 'taker': 0.0005},     # 0.02% / 0.05%
    'gateio_spot': {'maker': 0.001, 'taker': 0.001},     # 0.10% / 0.10%
    'gateio_futures': {'maker': 0.0002, 'taker': 0.0005} # 0.02% / 0.05%
}

# For delta-neutral arbitrage (spot + futures)
spot_fee = FEES['mexc_spot']['taker']  # MEXC spot trading
fut_fee = FEES['gateio_futures']['taker']  # GATEIO futures hedging

print("💰 Fee Configuration:")
print(f"  MEXC Spot Taker Fee: {spot_fee*100:.3f}%")
print(f"  GATEIO Futures Taker Fee: {fut_fee*100:.3f}%")
print(f"  Total Round-trip Fees: {(spot_fee + fut_fee)*2*100:.3f}%")
print(f"  Minimum Profitable Spread: >{(spot_fee + fut_fee)*2*100:.3f}%")

## 3. Spread Analysis

Analyze the statistical properties of spreads to understand market behavior.

In [ ]:
# Calculate spread statistics
spread_stats = calculate_spread_statistics(df, spot_fee, fut_fee)

print("📊 SPREAD STATISTICS ANALYSIS")
print("=" * 50)
print(f"Mean Entry Cost: {spread_stats.mean_entry_cost:.4f}%")
print(f"Std Entry Cost: {spread_stats.std_entry_cost:.4f}%")
print(f"Mean Reversion Speed: {spread_stats.mean_reversion_speed:.4f}")
print(f"Volatility Clustering: {spread_stats.volatility_clustering:.4f}")
print(f"Transaction Cost Floor: {spread_stats.transaction_cost_floor:.4f}%")

print("\n📈 Spread Percentiles:")
for percentile, value in spread_stats.percentiles.items():
    print(f"  {percentile:2d}th percentile: {value:8.4f}%")

In [ ]:
# Visualize spread distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle(f'Spread Analysis for {symbol.base}/{symbol.quote}', fontsize=16, fontweight='bold')

# Calculate spreads for visualization
df_analysis = df.copy()
df_analysis['entry_cost_pct'] = ((df_analysis['spot_ask_price'] - df_analysis['fut_bid_price']) / 
                                 df_analysis['spot_ask_price']) * 100
df_analysis['exit_cost_pct'] = ((df_analysis['fut_ask_price'] - df_analysis['spot_bid_price']) / 
                               df_analysis['fut_ask_price']) * 100

# 1. Entry cost distribution
axes[0,0].hist(df_analysis['entry_cost_pct'].dropna(), bins=50, alpha=0.7, edgecolor='black')
axes[0,0].axvline(spread_stats.transaction_cost_floor, color='red', linestyle='--', 
                  label=f'Min Profitable: {spread_stats.transaction_cost_floor:.3f}%')
axes[0,0].axvline(spread_stats.percentiles[20], color='green', linestyle='--', 
                  label=f'20th Percentile: {spread_stats.percentiles[20]:.3f}%')
axes[0,0].set_title('Entry Cost Distribution')
axes[0,0].set_xlabel('Entry Cost (%)')
axes[0,0].set_ylabel('Frequency')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. Time series of entry costs
axes[0,1].plot(df_analysis.index, df_analysis['entry_cost_pct'], alpha=0.7, linewidth=0.8)
axes[0,1].axhline(spread_stats.transaction_cost_floor, color='red', linestyle='--', alpha=0.8)
axes[0,1].axhline(spread_stats.percentiles[20], color='green', linestyle='--', alpha=0.8)
axes[0,1].set_title('Entry Cost Over Time')
axes[0,1].set_xlabel('Time')
axes[0,1].set_ylabel('Entry Cost (%)')
axes[0,1].grid(True, alpha=0.3)

# 3. Exit cost distribution
axes[1,0].hist(df_analysis['exit_cost_pct'].dropna(), bins=50, alpha=0.7, edgecolor='black', color='orange')
axes[1,0].axvline(0, color='black', linestyle='-', alpha=0.5, label='Break-even')
axes[1,0].set_title('Exit Cost Distribution')
axes[1,0].set_xlabel('Exit Cost (%)')
axes[1,0].set_ylabel('Frequency')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# 4. Spread correlation analysis
valid_data = df_analysis[['entry_cost_pct', 'exit_cost_pct']].dropna()
axes[1,1].scatter(valid_data['entry_cost_pct'], valid_data['exit_cost_pct'], 
                  alpha=0.5, s=1, color='purple')
axes[1,1].set_title('Entry vs Exit Cost Correlation')
axes[1,1].set_xlabel('Entry Cost (%)')
axes[1,1].set_ylabel('Exit Cost (%)')
axes[1,1].grid(True, alpha=0.3)

# Add correlation coefficient
correlation = valid_data['entry_cost_pct'].corr(valid_data['exit_cost_pct'])
axes[1,1].text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
               transform=axes[1,1].transAxes, bbox=dict(boxstyle="round", facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

print(f"\n📊 Key Observations:")
print(f"  Entry costs above minimum profitable: {(df_analysis['entry_cost_pct'] > spread_stats.transaction_cost_floor).mean()*100:.1f}%")
print(f"  Entry costs below 20th percentile: {(df_analysis['entry_cost_pct'] <= spread_stats.percentiles[20]).mean()*100:.1f}%")
print(f"  Favorable exit opportunities: {(df_analysis['exit_cost_pct'] < 0).mean()*100:.1f}%")

## 4. Parameter Optimization Comparison

Compare different optimization approaches to find the best parameters.

In [ ]:
# Run all optimization approaches
print("🔧 Running Parameter Optimization...")
print("This may take a few minutes for risk-adjusted optimization.")

# Statistical approaches (fast)
print("\n1️⃣ Statistical Approach...")
stat_result = optimize_parameters_statistical(df, spot_fee, fut_fee, conservatism_level='moderate')

print("2️⃣ Fast Statistical Approach...")
fast_stat_result = optimize_parameters_statistical_fast(df, spot_fee, fut_fee)

print("3️⃣ Random Sampling Approach...")
random_result = optimize_parameters_random_sampling(df, spot_fee, fut_fee, n_samples=20)

print("4️⃣ Risk-Adjusted Approach (may take longer)...")
try:
    risk_result = optimize_parameters_risk_adjusted(df, spot_fee, fut_fee)
    risk_available = True
except Exception as e:
    print(f"⚠️ Risk-adjusted optimization failed: {e}")
    print("Using statistical fallback...")
    risk_result = stat_result
    risk_available = False

print("✅ All optimizations completed!")

In [ ]:
# Create comparison summary
results_df = pd.DataFrame({
    'Statistical': [
        stat_result.max_entry_cost_pct,
        stat_result.min_profit_pct,
        stat_result.expected_trades_per_day,
        stat_result.expected_win_rate,
        stat_result.expected_avg_profit,
        stat_result.confidence_score
    ],
    'Fast Statistical': [
        fast_stat_result.max_entry_cost_pct,
        fast_stat_result.min_profit_pct,
        fast_stat_result.expected_trades_per_day,
        fast_stat_result.expected_win_rate,
        fast_stat_result.expected_avg_profit,
        fast_stat_result.confidence_score
    ],
    'Random Sampling': [
        random_result.max_entry_cost_pct,
        random_result.min_profit_pct,
        random_result.expected_trades_per_day,
        random_result.expected_win_rate,
        random_result.expected_avg_profit,
        random_result.confidence_score
    ],
    'Risk-Adjusted': [
        risk_result.max_entry_cost_pct,
        risk_result.min_profit_pct,
        risk_result.expected_trades_per_day,
        risk_result.expected_win_rate,
        risk_result.expected_avg_profit,
        risk_result.confidence_score
    ]
}, index=[
    'Entry Threshold (%)',
    'Exit Target (%)',
    'Trades/Day',
    'Win Rate',
    'Avg Profit (%)',
    'Confidence Score'
])

print("📊 PARAMETER OPTIMIZATION COMPARISON")
print("=" * 80)
display(results_df.round(4))

In [ ]:
# Visualize parameter comparison
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Parameter Optimization Comparison', fontsize=16, fontweight='bold')

methods = results_df.columns
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

# 1. Entry Threshold Comparison
bars1 = axes[0,0].bar(methods, results_df.loc['Entry Threshold (%)'], color=colors)
axes[0,0].axhline(spread_stats.transaction_cost_floor, color='red', linestyle='--', 
                  label=f'Min Required: {spread_stats.transaction_cost_floor:.3f}%')
axes[0,0].set_title('Entry Threshold Comparison')
axes[0,0].set_ylabel('Entry Threshold (%)')
axes[0,0].legend()
axes[0,0].tick_params(axis='x', rotation=45)

# 2. Exit Target Comparison
bars2 = axes[0,1].bar(methods, results_df.loc['Exit Target (%)'], color=colors)
axes[0,1].set_title('Exit Target Comparison')
axes[0,1].set_ylabel('Exit Target (%)')
axes[0,1].tick_params(axis='x', rotation=45)

# 3. Expected Trades per Day
bars3 = axes[0,2].bar(methods, results_df.loc['Trades/Day'], color=colors)
axes[0,2].set_title('Expected Daily Trade Frequency')
axes[0,2].set_ylabel('Trades per Day')
axes[0,2].tick_params(axis='x', rotation=45)

# 4. Win Rate Comparison
bars4 = axes[1,0].bar(methods, results_df.loc['Win Rate'], color=colors)
axes[1,0].set_title('Expected Win Rate')
axes[1,0].set_ylabel('Win Rate')
axes[1,0].set_ylim(0, 1)
axes[1,0].tick_params(axis='x', rotation=45)

# 5. Average Profit Comparison
bars5 = axes[1,1].bar(methods, results_df.loc['Avg Profit (%)'], color=colors)
axes[1,1].set_title('Expected Average Profit')
axes[1,1].set_ylabel('Average Profit (%)')
axes[1,1].tick_params(axis='x', rotation=45)

# 6. Confidence Score
bars6 = axes[1,2].bar(methods, results_df.loc['Confidence Score'], color=colors)
axes[1,2].set_title('Confidence Score')
axes[1,2].set_ylabel('Confidence Score')
axes[1,2].set_ylim(0, 1)
axes[1,2].tick_params(axis='x', rotation=45)

# Add value labels on bars
for ax, metric in zip(axes.flat, results_df.index):
    for i, bar in enumerate(ax.patches):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## 5. Detailed Analysis of Best Approach

Select and analyze the most promising parameter set.

In [ ]:
# Select best approach based on confidence score and practical metrics
approach_scores = {
    'Statistical': stat_result.confidence_score * stat_result.expected_win_rate,
    'Fast Statistical': fast_stat_result.confidence_score * fast_stat_result.expected_win_rate,
    'Random Sampling': random_result.confidence_score * random_result.expected_win_rate,
    'Risk-Adjusted': risk_result.confidence_score * risk_result.expected_win_rate
}

best_approach = max(approach_scores, key=approach_scores.get)
approach_mapping = {
    'Statistical': stat_result,
    'Fast Statistical': fast_stat_result,
    'Random Sampling': random_result,
    'Risk-Adjusted': risk_result
}
best_result = approach_mapping[best_approach]

print(f"🏆 BEST APPROACH: {best_approach}")
print(f"📊 Combined Score: {approach_scores[best_approach]:.4f}")
print("=" * 60)
print(f"Entry Threshold: {best_result.max_entry_cost_pct:.4f}%")
print(f"Exit Target: {best_result.min_profit_pct:.4f}%")
print(f"Expected Trades/Day: {best_result.expected_trades_per_day:.2f}")
print(f"Expected Win Rate: {best_result.expected_win_rate:.1%}")
print(f"Expected Avg Profit: {best_result.expected_avg_profit:.4f}%")
print(f"Confidence Score: {best_result.confidence_score:.3f}")
print(f"\n💡 Reasoning: {best_result.reasoning}")

## 6. Sensitivity Analysis

Test how parameter performance changes with different fee structures and market conditions.

In [ ]:
# Fee sensitivity analysis
fee_scenarios = {
    'Low Fees (VIP)': {'spot': 0.0001, 'fut': 0.0001},
    'Current Fees': {'spot': spot_fee, 'fut': fut_fee},
    'High Fees (Retail)': {'spot': 0.001, 'fut': 0.001},
    'Very High Fees': {'spot': 0.002, 'fut': 0.002}
}

sensitivity_results = {}

print("🔬 Fee Sensitivity Analysis")
print("=" * 50)

for scenario_name, fees in fee_scenarios.items():
    try:
        result = optimize_parameters_statistical_fast(
            df, 
            spot_fee=fees['spot'], 
            fut_fee=fees['fut']
        )
        sensitivity_results[scenario_name] = result
        
        total_fees = (fees['spot'] + fees['fut']) * 2 * 100
        print(f"\n{scenario_name} (Total: {total_fees:.3f}%):")
        print(f"  Entry: {result.max_entry_cost_pct:.4f}%, Exit: {result.min_profit_pct:.4f}%")
        print(f"  Trades/Day: {result.expected_trades_per_day:.2f}, Win Rate: {result.expected_win_rate:.1%}")
        
    except Exception as e:
        print(f"❌ Error in {scenario_name}: {e}")
        continue

print("\n✅ Sensitivity analysis completed")

In [ ]:
# Visualize fee sensitivity
if sensitivity_results:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Fee Sensitivity Analysis', fontsize=16, fontweight='bold')
    
    scenarios = list(sensitivity_results.keys())
    colors = plt.cm.viridis(np.linspace(0, 1, len(scenarios)))
    
    # Extract data for plotting
    entry_thresholds = [sensitivity_results[s].max_entry_cost_pct for s in scenarios]
    exit_targets = [sensitivity_results[s].min_profit_pct for s in scenarios]
    trades_per_day = [sensitivity_results[s].expected_trades_per_day for s in scenarios]
    win_rates = [sensitivity_results[s].expected_win_rate for s in scenarios]
    
    # Fee levels for x-axis
    fee_levels = [(fee_scenarios[s]['spot'] + fee_scenarios[s]['fut']) * 2 * 100 for s in scenarios]
    
    # Plot relationships
    axes[0,0].plot(fee_levels, entry_thresholds, 'o-', linewidth=2, markersize=8)
    axes[0,0].set_title('Entry Threshold vs Total Fees')
    axes[0,0].set_xlabel('Total Round-trip Fees (%)')
    axes[0,0].set_ylabel('Entry Threshold (%)')
    axes[0,0].grid(True, alpha=0.3)
    
    axes[0,1].plot(fee_levels, exit_targets, 'o-', linewidth=2, markersize=8, color='orange')
    axes[0,1].set_title('Exit Target vs Total Fees')
    axes[0,1].set_xlabel('Total Round-trip Fees (%)')
    axes[0,1].set_ylabel('Exit Target (%)')
    axes[0,1].grid(True, alpha=0.3)
    
    axes[1,0].plot(fee_levels, trades_per_day, 'o-', linewidth=2, markersize=8, color='green')
    axes[1,0].set_title('Trade Frequency vs Total Fees')
    axes[1,0].set_xlabel('Total Round-trip Fees (%)')
    axes[1,0].set_ylabel('Trades per Day')
    axes[1,0].grid(True, alpha=0.3)
    
    axes[1,1].plot(fee_levels, win_rates, 'o-', linewidth=2, markersize=8, color='red')
    axes[1,1].set_title('Win Rate vs Total Fees')
    axes[1,1].set_xlabel('Total Round-trip Fees (%)')
    axes[1,1].set_ylabel('Expected Win Rate')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n📈 Key Insights:")
    print(f"  Fee impact on entry threshold: {(max(entry_thresholds) - min(entry_thresholds)):.4f}% range")
    print(f"  Fee impact on trade frequency: {(max(trades_per_day) - min(trades_per_day)):.2f} trades/day range")
    print(f"  Most sensitive metric: Trade frequency (varies with fee structure)")

## 7. Implementation Recommendations

Final recommendations for implementing the optimized parameters.

In [ ]:
# Generate implementation recommendations
print("🎯 IMPLEMENTATION RECOMMENDATIONS")
print("=" * 60)

print(f"\n🔧 OPTIMAL PARAMETERS ({best_approach} Approach):")
print(f"  max_entry_cost_pct = {best_result.max_entry_cost_pct:.4f}")
print(f"  min_profit_pct = {best_result.min_profit_pct:.4f}")
print(f"  max_hours = 6  # Standard timeout")
print(f"  spot_fee = {spot_fee:.4f}")
print(f"  fut_fee = {fut_fee:.4f}")

print(f"\n📊 EXPECTED PERFORMANCE:")
print(f"  Daily trade frequency: {best_result.expected_trades_per_day:.1f} trades")
print(f"  Win rate: {best_result.expected_win_rate:.1%}")
print(f"  Average profit per trade: {best_result.expected_avg_profit:.3f}%")
print(f"  Confidence level: {best_result.confidence_score:.1%}")

# Calculate potential returns
daily_return = best_result.expected_avg_profit * best_result.expected_trades_per_day * best_result.expected_win_rate / 100
monthly_return = daily_return * 22  # 22 trading days per month
annual_return = daily_return * 252  # 252 trading days per year

print(f"\n💰 PROJECTED RETURNS:")
print(f"  Daily return: {daily_return:.4f}%")
print(f"  Monthly return: {monthly_return:.3f}%")
print(f"  Annual return: {annual_return:.2f}%")

print(f"\n⚠️  RISK CONSIDERATIONS:")
print(f"  • Parameters optimized on historical data - future performance may vary")
print(f"  • Market conditions can change - monitor and re-optimize regularly")
print(f"  • Fee structures may change - sensitivity analysis shows impact")
print(f"  • Consider starting with conservative position sizes")
print(f"  • Implement proper risk management and stop-losses")

print(f"\n🔄 MONITORING RECOMMENDATIONS:")
print(f"  • Re-optimize parameters weekly with fresh data")
print(f"  • Track actual vs expected performance metrics")
print(f"  • Monitor spread characteristics for regime changes")
print(f"  • Adjust parameters if win rate drops below {best_result.expected_win_rate*0.8:.1%}")

print(f"\n📝 PYTHON IMPLEMENTATION CODE:")
print(f"```python")
print(f"# Optimized parameters for {symbol.base}/{symbol.quote}")
print(f"trades = delta_neutral_backtest(")
print(f"    df,")
print(f"    max_entry_cost_pct={best_result.max_entry_cost_pct:.4f},")
print(f"    min_profit_pct={best_result.min_profit_pct:.4f},")
print(f"    max_hours=6,")
print(f"    spot_fee={spot_fee:.4f},")
print(f"    fut_fee={fut_fee:.4f}")
print(f")")
print(f"```")

## 8. Summary and Next Steps

Final summary of the optimization analysis.

In [ ]:
# Create final summary table
summary_data = {
    'Metric': [
        'Symbol',
        'Data Period',
        'Data Points',
        'Best Approach',
        'Entry Threshold',
        'Exit Target',
        'Expected Trades/Day',
        'Expected Win Rate',
        'Expected Daily Return',
        'Annual Return Projection',
        'Confidence Score'
    ],
    'Value': [
        f"{symbol.base}/{symbol.quote}",
        f"{(date_to - date_from).total_seconds() / 3600:.1f} hours",
        f"{len(df):,}",
        best_approach,
        f"{best_result.max_entry_cost_pct:.4f}%",
        f"{best_result.min_profit_pct:.4f}%",
        f"{best_result.expected_trades_per_day:.2f}",
        f"{best_result.expected_win_rate:.1%}",
        f"{daily_return:.4f}%",
        f"{annual_return:.2f}%",
        f"{best_result.confidence_score:.3f}"
    ]
}

summary_df = pd.DataFrame(summary_data)

print("📋 OPTIMIZATION SUMMARY")
print("=" * 40)
display(summary_df)

print("\n🎉 Parameter optimization analysis completed successfully!")
print("\n📖 Next Steps:")
print("  1. Implement the optimized parameters in your trading system")
print("  2. Start with paper trading to validate performance")
print("  3. Monitor actual vs expected metrics")
print("  4. Re-run optimization weekly with fresh data")
print("  5. Consider testing on other trading pairs")

# Save results for future reference
import json
optimization_results = {
    'symbol': f"{symbol.base}/{symbol.quote}",
    'best_approach': best_approach,
    'parameters': {
        'max_entry_cost_pct': best_result.max_entry_cost_pct,
        'min_profit_pct': best_result.min_profit_pct,
        'spot_fee': spot_fee,
        'fut_fee': fut_fee
    },
    'performance': {
        'expected_trades_per_day': best_result.expected_trades_per_day,
        'expected_win_rate': best_result.expected_win_rate,
        'expected_avg_profit': best_result.expected_avg_profit,
        'confidence_score': best_result.confidence_score
    },
    'timestamp': datetime.now().isoformat()
}

# Save to file
output_file = f'/Users/dasein/dev/cex_arbitrage/src/trading/research/optimization_results_{symbol.base}_{symbol.quote}.json'
with open(output_file, 'w') as f:
    json.dump(optimization_results, f, indent=2)

print(f"\n💾 Results saved to: {output_file}")